In [1]:
import numpy as np
import pandas as pd

**Zadanie 1**  
Wczytaj plik `zamowienia.csv` do ramki pandas, a następnie w kilku miejscach (ale nie w pierwszych 10 wierszach) wstaw wartość NaN, aby zasymulować wartości brakujące. Zapisz ramkę do pliku `zamowienia_missing.csv`. Wczytaj teraz plik do ramki Dask i sprawdź jakie typy danych zostały przydzielone. Czy zgadzają się z typami z oryginalnego pliku? Wykonaj dowolne obliczenia na całej ramce Dask, aby wymusić wywołanie `.compute()`. Czy pojawił się błąd dotyczący niespójności typów danych? Spróbuj uruchomić kilka razy funkcję wczytywania danych do ramki Dask dataframe z różnymi wartościami parametru `samples`. Dokumentacja `dask.dataframe.read_csv()`: https://docs.dask.org/en/stable/generated/dask.dataframe.read_csv.html

In [2]:
df = pd.read_csv('../Lab01 - scattered processing and optimalization/zamowienia.csv', header=0, sep=';')
df.head()

,Kraj,Sprzedawca,Data zamowienia,idZamowienia,Utarg
0,Polska,Kowalski,2003-07-16,10248,440.00
1,Polska,Sowiński,2003-07-10,10249,1863.40
2,Niemcy,Peacock,2003-07-12,10250,1552.60
3,Niemcy,Leverling,2003-07-15,10251,654.06
4,Niemcy,Peacock,2003-07-11,10252,3597.90


In [3]:
df_with_nan = df.copy()
nan_fields = [(16, 'Utarg'), (24, 'Sprzedawca'), (29, 'Data zamowienia'), (33, 'Kraj')]

for index, column in nan_fields:
    df_with_nan.at[index, column] = np.nan

df_with_nan.to_csv('./data/zamowienia_missing.csv', index=False)

df_with_nan.loc[nan_fields[0][0]:nan_fields[-1][0]]

,Kraj,Sprzedawca,Data zamowienia,idZamowienia,Utarg
16,Polska,Sowiński,2003-08-23,10264,NaN
17,Niemcy,Fuller,2003-08-12,10265,1176.00
18,Niemcy,Leverling,2003-07-31,10266,346.56
19,Niemcy,Peacock,2003-08-06,10267,3536.60
20,Niemcy,Callahan,2003-08-02,10268,1101.20
21,Polska,Kowalski,2003-08-09,10269,642.20
22,Niemcy,Davolio,2003-08-02,10270,1376.00
23,Polska,Sowiński,2003-08-30,10271,48.00
24,Polska,NaN,2003-08-06,10272,1456.00
25,Niemcy,Leverling,2003-08-12,10273,2037.28


In [4]:
import dask.dataframe as dd

dask_df = dd.read_csv('./data/zamowienia_missing.csv', delimiter=',')

print(dask_df.dtypes)

Kraj               string[pyarrow]
Sprzedawca         string[pyarrow]
Data zamowienia    string[pyarrow]
idZamowienia                 int64
Utarg                      float64
dtype: object


In [5]:
print(df.dtypes)

Kraj                object
Sprzedawca          object
Data zamowienia     object
idZamowienia         int64
Utarg              float64
dtype: object


Typy się nie zgadzają, zostały zmienione z object na string[pyarrow]

In [6]:
result = dask_df['Kraj'].mode().compute()
print(result)

0    Niemcy
Name: Kraj, dtype: object


In [7]:
result = dask_df['Utarg'].mean().compute()
print(result)

1538.3856892230574


Brak błędu niespójności danych.

**Zadanie 2**  
Ze strony https://docs.dask.org/en/stable/dashboard.html skonfiguruj plugin Dask dashboard dla Jupyter Lab i przetestuj jego działanie.

http://localhost:8888/lab/tree/Lab02%20-%20dask/lab_02.ipynb tu działa jako tako

**Zadanie 3**  
Skonfiguruj lokalny klaster (`Client`) tak, aby nie zaalokował wszystkich zasobów (np. zostaw 8 GB RAM dla systemu hosta + 2 rdzenie). Pobierz dane udostępnione na poprzednich zajęciach (https://huggingface.co/datasets/vargr/private_instagram/tree/main/data) i załaduj do ramki Dask tyle części ile zdołasz w formie bez optymalizacji. Zmierz czas tej operacji. 

In [8]:
from dask.distributed import Client
import glob
import time

client = Client(memory_limit='24GB', n_workers=12)

path = '../data/Lab01/*.parquet'
files = glob.glob(path)

start_time = time.time()
df = dd.concat([dd.read_parquet(file) for file in files], ignore_index=True)
end_time = time.time()
loading_time = end_time - start_time

print(f'Czas ładowania danych: {loading_time} sekund')

Czas ładowania danych: 0.6707925796508789 sekund


**Zadanie 4**  
Wykonaj kilka operacji na klastrze lokalnym z danymi z zadania 3:
* wyświetl top 10 użytkowników z najwyższą liczbą like'ów,
* pobierz dane tylko za pierwsze półrocze 2019 roku.
Każdorazowo zmierz i wyświetl czas operacji i obserwuj dashboard.

In [9]:
start_time = time.time()
top_users = df.nlargest(10, 'likes')[['sid_profile', 'likes']].compute()
end_time = time.time()
operation_time = end_time - start_time

print(top_users)
print(f'Czas operacji top 10 użytkowników: {operation_time} sekund')

        sid_profile    likes
263914       531404  8822952
263910       531404  5447066
263911       531404  5116398
263908       531404  4271466
263915       531404  3558599
384081       397587  3533326
592831      3520880  3340297
115579        91738  3311246
263917       531404  3198242
263909       531404  2900455
Czas operacji top 10 użytkowników: 1.19858717918396 sekund


In [10]:
start_time = time.time()
filtered_df = df[df['date'].between('2019-01-01', '2019-06-30')].compute()
end_time = time.time()
filter_time = end_time - start_time

print(filtered_df.head())
print(f'Czas filtrowania danych: {filter_time} sekund')

        sid  sid_profile      post_id  profile_id                 date  \
2  28370905      3496776  Bunhd1DFVAG  2237947779  2019-03-05 08:03:11   
4  32170690      3496776  BuDfIyslzfw  2237947779  2019-02-19 08:10:11   
5  14315358      3496776  BxJsMDpA2yH  2237947779  2019-05-07 08:33:51   
6   8304346      3496776  Bt5LFpZlm3z  2237947779  2019-02-15 08:02:35   
7  14315346      3496776  BxZIzaQhS-o  2237947779  2019-05-13 08:32:30   

   post_type                                        description  likes  \
2          1  Tech Tuesday. Been flat out on the tools. Got ...    168   
4          1  Solid effort on the bar turn.\nFully turned.\n...    145   
5          1  Annual springtime flora picture.\nTurn bars in...    124   
6          1  Laps in spring like conditions. Getting these ...    150   
7          1  Cheers Scotland 🏴󠁧󠁢󠁳󠁣󠁴󠁿 See you in a few weeks...    166   

   comments   username                                                bio  \
2         3  andylund_  Professio

**Zadanie 5**  
Wczytaj te same dane do ramki Dask co w zadaniu 3, ale podaj typy danych, które zostały wybrane w procesie optymalizacji wykonanej w zadaniach z lab 01. Porównaj czas ładowania z zadaniem 3. Wykonaj również te same operacje co w zadaniu 4 i porównaj czas. Śledź wykonanie zadań patrząć na graf wywołań.

In [11]:
start_time = time.time()

df2 = df.copy()

df2['sid'] = df['sid'].astype(np.int32)
df2['sid_profile'] = df['sid_profile'].astype(np.int32)
df2['post_id'] = df['post_id']
df2['profile_id'] = df['profile_id']
df2['date'] = dd.to_datetime(df['date'])
df2['post_type'] = df['post_type'].astype('category')
df2['description'] = df['description']
df2['likes'] = df['likes'].astype(np.int32)
df2['comments'] = df['comments'].astype(np.int32)
df2['username'] = df['username']
df2['bio'] = df['bio']
df2['following'] = df['following'].astype(np.int32)
df2['followers'] = df['followers'].astype(np.int32)
df2['num_posts'] = df['num_posts'].astype(np.int32)
df2['is_business_account'] = df['is_business_account']
df2['lang'] = df['lang'].astype('category')
df2['category'] = df['category'].astype('category')

end_time = time.time()
optimized_loading_time = end_time - start_time

print(f'Czas ładowania z optymalizacją: {optimized_loading_time} sekund')

Czas ładowania z optymalizacją: 0.031199216842651367 sekund


In [12]:
start_time = time.time()
top_users = df2.nlargest(10, 'likes')[['sid_profile', 'likes']].compute()
end_time = time.time()
operation_time = end_time - start_time

print(top_users)
print(f'Czas operacji top 10 użytkowników: {operation_time} sekund')

        sid_profile    likes
263914       531404  8822952
263910       531404  5447066
263911       531404  5116398
263908       531404  4271466
263915       531404  3558599
384081       397587  3533326
592831      3520880  3340297
115579        91738  3311246
263917       531404  3198242
263909       531404  2900455
Czas operacji top 10 użytkowników: 3.401810884475708 sekund


In [13]:
start_time = time.time()
filtered_df = df2[df2['date'].between('2019-01-01', '2019-06-30')].compute()
end_time = time.time()
filter_time = end_time - start_time

print(filtered_df.head())
print(f'Czas filtrowania danych: {filter_time} sekund')

        sid  sid_profile      post_id  profile_id                date  \
2  28370905      3496776  Bunhd1DFVAG  2237947779 2019-03-05 08:03:11   
4  32170690      3496776  BuDfIyslzfw  2237947779 2019-02-19 08:10:11   
5  14315358      3496776  BxJsMDpA2yH  2237947779 2019-05-07 08:33:51   
6   8304346      3496776  Bt5LFpZlm3z  2237947779 2019-02-15 08:02:35   
7  14315346      3496776  BxZIzaQhS-o  2237947779 2019-05-13 08:32:30   

  post_type                                        description  likes  \
2         1  Tech Tuesday. Been flat out on the tools. Got ...    168   
4         1  Solid effort on the bar turn.\nFully turned.\n...    145   
5         1  Annual springtime flora picture.\nTurn bars in...    124   
6         1  Laps in spring like conditions. Getting these ...    150   
7         1  Cheers Scotland 🏴󠁧󠁢󠁳󠁣󠁴󠁿 See you in a few weeks...    166   

   comments   username                                                bio  \
2         3  andylund_  Professional Bicycle 

**Zadanie 6**  
Podziel tablicę `darr` z przykładów na inne liczby chunków (eksperymentuj) i wykonaj te same obliczenie (średnia). Dla każdej liczby chunków wypisz czas obliczeń (wykonaj to samo obliczenie minimum 10 razy, aby nieco uwiarygodnić wyniki i uśrednij) i porównaj wyniki. Napisz wniosek o wynikach swoich eksperymentów i automatycznego podziału na chunki. Czy udało Ci się osiągnąć lepszą wydajność niż przy domyślnych ustawieniach?

In [15]:
import dask.array as da


def measure_mean_time(darr, n_trials=10):
    times = []
    for _ in range(n_trials):
        start_time = time.time()
        darr.mean(axis=0).compute()
        times.append(time.time() - start_time)
    return np.mean(times)


chunk_sizes = [
    (500, 500),
    (1000, 1000),
    (2000, 2000),
    (5000, 5000),
    (500, 5000),
    'auto'
]
n_trials = 10
results = {}

for chunks in chunk_sizes:
    if chunks == 'auto':
        darr = da.random.normal(5, 0.2, size=(20_000, 20_000))
    else:
        darr = da.random.normal(5, 0.2, size=(20_000, 20_000), chunks=chunks)
    
    avg_time = measure_mean_time(darr, n_trials)
    results[chunks] = avg_time
    print(f'Średni czas dla chunków {chunks}: {avg_time:.4f} sekund')

Średni czas dla chunków (500, 500): 2.3447 sekund
Średni czas dla chunków (1000, 1000): 1.0596 sekund
Średni czas dla chunków (2000, 2000): 0.8216 sekund
Średni czas dla chunków (5000, 5000): 0.8251 sekund
Średni czas dla chunków (500, 5000): 0.8831 sekund
Średni czas dla chunków auto: 0.1216 sekund


Średni czas dla chunków (500, 500): 2.7237 sekund  
Średni czas dla chunków (1000, 1000): 1.1550 sekund  
Średni czas dla chunków (2000, 2000): 0.8748 sekund  
Średni czas dla chunków (5000, 5000): 0.2675 sekund  
Średni czas dla chunków (500, 5000): 0.7797 sekund  
Średni czas dla chunków auto: 0.4135 sekund  

Im więcej chunków tym szybciej i udało się przebić domyślne ustawienia.